In [23]:
import pickle
import numpy as np
import pandas as pd
from time import time
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [24]:
with open('NHLegTimeSeriesData.dict','rb') as f:
    leg_data = pickle.load(f)
with open('NHLegTimeSeriesLabels.list','rb') as f:
    leg_target = pickle.load(f)

We are trying to predict the next state of the bill given current state, bill type, time, sponsors.
* We should save the probabilities of each class, then create a markov chain with them
* We should create a separate classifier for the raw beginning probability

In [12]:
print()
print(leg_target[0:10]) 
print(leg_data['bill_cat'][0:10]) #clustered bill type, 1-50 ish
print()
print(leg_data['sponsors'][0:10]) #republican or democrat, -1 or 1, democract is 1 and republican is -1 
print()
print(leg_data['event'][0:10])#
print()
print(leg_data['time_t'][0:10])
print()
print(len(leg_target)) #34014 state changes, not bills


['public hearing', 'executive session', 'committee report inexpedient to legislate vote', 'inexpedient to legislate motion adopted voice vote', 'inexpedient to legislate motion adopted voice vote', 'vacated to children and family law without objection', 'public hearing', 'executive session', 'committee report inexpedient to legislate vote', 'removed from consent']
[25, 25, 25, 25, 25, 25, 25, 25, 25, 25]

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

['introduced referred to legislative administration', 'public hearing', 'executive session', 'committee report inexpedient to legislate vote', 'inexpedient to legislate motion adopted voice vote', 'introduced referred to health human services elderly affairs', 'vacated to children and family law without objection', 'public hearing', 'executive session', 'committee report inexpedient to legislate vote']

[1, 2, 3, 4, 5, 1, 2, 3, 4, 5]

34014


In [25]:
leg_data_df = pd.DataFrame(leg_data)
leg_data_df = pd.get_dummies(leg_data_df)
leg_label_df = pd.DataFrame(leg_target)
leg_label_df = pd.get_dummies(leg_label_df)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(leg_data_df, leg_target, test_size=0.3,random_state=109) # 70% training and 30% test

In [12]:
#Create a Gaussian Classifier
mnnb = MultinomialNB()

#Train the model using the training sets
mnnb.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = mnnb.predict(X_test)
y_proba = mnnb.predict_proba(X_test)
y_proba_log = mnnb.predict_log_proba(X_test)

ValueError: Input X must be non-negative

In [27]:
#https://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html#sphx-glr-auto-examples-model-selection-plot-randomized-search-py

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_accuracy'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation accuracy: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_accuracy'][candidate],
                  results['std_test_accuracy'][candidate]))
            print('Precision: ', results['mean_test_precision_micro'][candidate],
                  'Recall: ', results['mean_test_recall_micro'][candidate],
                  'f1: ', results['mean_test_f1_micro'][candidate])
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [28]:
#Create a Gaussian Classifier
tree = DecisionTreeClassifier()#criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)

# specify parameters and distributions to sample from
param_dist = {"max_depth": [100, None],
              "min_samples_split": sp_randint(2, 11),
              "max_leaf_nodes":[100, 1000, None],
              "splitter":['best', 'random'],
              "criterion":["gini", "entropy"]}

n_iter_search = 20
#f1_micro because multiclass and unbalanced,
random_search = RandomizedSearchCV(tree, scoring =['accuracy', 'f1_micro', 'precision_micro', 'recall_micro'], param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, iid=False, refit='accuracy')

start = time()
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

/usr/local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


KeyboardInterrupt: 

In [40]:
print(random_search.cv_results_)
#report(random_search.cv_results_)

{'split0_test_precision_micro': array([0.18895706, 0.40961145, 0.21206544, 0.51635992, 0.30674847,
       0.33047035, 0.18875256, 0.41186094, 0.21206544, 0.29325153,
       0.40695297, 0.39693252, 0.40920245, 0.36462168, 0.45787321,
       0.43803681, 0.51860941, 0.30838446, 0.19672802, 0.42597137]), 'mean_fit_time': array([0.29254336, 0.75568728, 0.30418406, 1.46630878, 0.43707218,
       0.61256413, 0.28999281, 0.92161541, 0.2955543 , 0.5124815 ,
       0.78133545, 0.85735712, 0.96868219, 0.63943205, 1.12966394,
       1.19352283, 1.32101979, 0.43397546, 0.2546608 , 1.08405795]), 'std_score_time': array([0.00736675, 0.01014726, 0.00563687, 0.00728484, 0.00645358,
       0.00933725, 0.01682207, 0.00634147, 0.01082051, 0.00572991,
       0.0103754 , 0.00619462, 0.00778437, 0.0192666 , 0.00685652,
       0.00399626, 0.01707857, 0.00663678, 0.01876013, 0.01669023]), 'split2_train_recall_micro': array([0.19698877, 0.48520617, 0.18854265, 0.80820481, 0.32163467,
       0.33506453, 0.196936

In [ ]:
#Train the model using the training sets
#tree.fit(X_train, y_train)
tree = DecisionTreeClassifier(max_depth= 100, min_samples_split= 5, max_leaf_nodes= 1000, criterion= 'gini', splitter= 'random')
tree.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = tree.predict(X_test)
y_proba = tree.predict_proba(X_test)
#y_proba_log = tree.predict_log_proba(X_test)

In [10]:
num_to_check = 0
print(len(y_proba[num_to_check]))
print(y_proba[num_to_check])

261
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.00214133 0.99357602
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        

In [11]:
x=0
y=0

for i in range(10205):
    if y_test[i] == y_pred[i]:
        x+=1
    y+=1

print(x,y,len(y_pred),len(y_test))

5584 10205 10205 10205
